# 🧑‍🔬 Лабораторная работа №5
## 🙋 Пикалов Николай Николаевич
## 📚 ПИ21-7
## 📅 06.12.2023

## 🔰 Задание 1 Изучите технологии attention и архитектуры нейронных сетей трансформеров.

## Технология Attention и Архитектура Трансформеров 🤖📘

### Технология Attention 👀

- **Определение**: Attention — это механизм в нейросетях, позволяющий модели сфокусироваться на определенных частях входных данных, что улучшает качество обработки естественного языка и перевода.
- **Функционал**: Позволяет модели определять важность различных частей входных данных, например, слов в предложении, для более точного вывода.
- **Применение**: Особенно полезен в задачах обработки естественного языка, таких как машинный перевод, генерация текста, распознавание речи)

## Архитектура Трансформеров ⚙️📚

- **Основа**: Трансформеры — это тип архитектуры нейронных сетей, основанный на полностью связанных слоях без использования рекуррентных связей.
- **Состав**: Архитектура включает в себя блоки кодировщика и декодировщика. Каждый блок состоит из подслоев, таких как:
  - **Многослойные перцептроны (MLP)**: Используются для обработки информации внутри каждого блока.
  - **Механизмы самовнимания (Self-Attention)**: Позволяют каждому элементу входных данных взаимодействовать с другими элементами для получения более глубокого понимания контекста.
- **Преимущества**: Трансформеры обеспечивают высокую скорость обработки и точность, особенно для длинных последовательностей данных.

## 🔰 Задание 2 
### 🗨️ Примените один из трансформеров, например BERT к задаче классификации отзывов клиентов.
### 🗨️ Сравните полученные результаты классическими методами машинного обучения, с RNN.
### 🗨️ Сделайте выводы.

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification
from keras.preprocessing.sequence import pad_sequences

In [6]:
# Загрузка данных
df = pd.read_csv('data/amazon_reviews.csv')
df = df[['reviews.text', 'reviews.rating']]  # Оставляем только необходимые колонки
df.dropna(inplace=True)  # Удаляем пропущенные значения

In [7]:
# Предобработка данных
# Преобразуем рейтинги в бинарные метки (например, положительный отзыв = 1, отрицательный = 0)
df['reviews.rating'] = df['reviews.rating'].apply(lambda x: 1 if x > 3 else 0)

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(df['reviews.text'], df['reviews.rating'], test_size=0.2, random_state=42)

## Классический метод ML (TF-IDF + Логистическая регрессия)

In [26]:
# Векторизация текстов
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Обучение модели
model_lr = LogisticRegression()
model_lr.fit(X_train_tfidf, y_train)

# Оценка модели
predictions_lr = model_lr.predict(X_test_tfidf)
print(classification_report(y_test, predictions_lr))


              precision    recall  f1-score   support

           0       0.88      0.40      0.55       579
           1       0.94      0.99      0.96      5088

    accuracy                           0.93      5667
   macro avg       0.91      0.70      0.76      5667
weighted avg       0.93      0.93      0.92      5667



## 📊 Отчёт по TF-IDF + Логистическая регрессия

- ✅ **Положительные отзывы (класс 1):**
  - Модель показывает **высокую точность и полноту** для класса 1, что означает эффективное определение положительных отзывов.

- ❌ **Негативные отзывы (класс 0):**
  - Производительность для класса 0 значительно **ниже**, предполагая трудности модели с идентификацией негативных отзывов.

- 🎯 **Общая точность:**
  - Модель достигает общей точности в **0.93**, что является довольно высоким показателем.

- 📉 **Средняя f1-оценка по макро:**
  - Среднее значение **0.76** может указывать на **различия между классами**.

- 🔍 **Взвешенная средняя f1-оценка:**
  - Значение в **0.92**, учитывающее поддержку, указывает на лучшую производительность модели на более частых метках.

## 📝 Выводы

- 🚀 **Модель логистической регрессии с TF-IDF** кажется **эффективной** на данном наборе данных, особенно для положительных отзывов.
- ⚖️ **Дисбаланс классов** или другие проблемы могут снижать эффективность модели для негативны негативных отзывов.
ках.

## RNN

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Параметры
max_len = 100 # Максимальная длина последовательности
max_words = 10000 # Размер словаря

# Инициализация и подгонка токенизатора
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train) # Предполагается, что X_train - это список текстов для обучения

# Преобразование текстов в последовательности индексов
sequences = tokenizer.texts_to_sequences(X_train)

# Паддинг последовательностей
X_train_pad = pad_sequences(sequences, maxlen=max_len)

In [7]:
# Создание модели RNN
model = Sequential()
model.add(Embedding(max_words, 32)) # Слой векторных представлений слов
model.add(SimpleRNN(32)) # RNN слой с 32 скрытыми состояниями
model.add(Dense(1, activation='sigmoid')) # Выходной слой с сигмоидной активацией для классификации

In [9]:
# Компиляция модели
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [10]:
# Обучение модели
model.fit(X_train_pad, y_train, epochs=20, batch_size=128, validation_split=0.2)

Epoch 1/20
142/142 [==============================] - 2s 10ms/step - loss: 0.0342 - acc: 0.9894 - val_loss: 0.1939 - val_acc: 0.9477
Epoch 2/20
142/142 [==============================] - 1s 9ms/step - loss: 0.0250 - acc: 0.9926 - val_loss: 0.2137 - val_acc: 0.9479
Epoch 3/20
142/142 [==============================] - 1s 10ms/step - loss: 0.0197 - acc: 0.9950 - val_loss: 0.2694 - val_acc: 0.9283
Epoch 4/20
142/142 [==============================] - 1s 10ms/step - loss: 0.0191 - acc: 0.9947 - val_loss: 0.2282 - val_acc: 0.9475
Epoch 5/20
142/142 [==============================] - 1s 10ms/step - loss: 0.0162 - acc: 0.9956 - val_loss: 0.2435 - val_acc: 0.9431
Epoch 6/20
142/142 [==============================] - 1s 10ms/step - loss: 0.0205 - acc: 0.9941 - val_loss: 0.2443 - val_acc: 0.9440
Epoch 7/20
142/142 [==============================] - 1s 10ms/step - loss: 0.0120 - acc: 0.9970 - val_loss: 0.2512 - val_acc: 0.9477
Epoch 8/20
142/142 [==============================] - 1s 10ms/step - l

## 📈 Анализ результатов обучения

- 🎯 **Точность на обучающем наборе данных (`acc`):**
  - Увеличивается с течением времени, достигая в последней эпохе **99.66%**.
  - Это указывает на способность модели корректно классифицировать данные, на которых она обучалась.

- 📉 **Функция потерь на обучающем наборе данных (`loss`):**
  - Уменьшается, что свидетельствует об улучшении производительности модели на обучающих данных.

- 🚫 **Точность на валидационном наборе данных (`val_acc`):**
  - Остается относительно стабильной после первой эпохи, в среднем около **94%**.
  - Отсутствие стабильного увеличения может указывать на то, что модель не улучшает свою способность обобщать данные.

- ⚠️ **Функция потерь на валидационном наборе данных (`val_loss`):**
  - Начинает увеличиваться после 10-й эпохи, что может быть признаком **переобучения**.

## 🔍 Выводы и следующие шаги

- 🛠️ **Противодействие переобучению:**
  - Рассмотреть использование методов регуляризации, таких как **Dropout**.
  - Применить **раннюю остановку** (Early Stopping) для предотвращения дальнейшего переобучения.
  - Использовать больший набор данных для валидации.

- 🧪 **Эксперименты с гиперпараметрами:**
  - Изменить количество эпох или размер пакета (batch size), чтобы найти оптимальный баланс между точностью и обобщающей способностью.

- 📊 **Балансировка производительности:**
  - Стремиться к улучшению производительности на валидационном наборе данных, чтобы модель лучше работала с новыми данными.
работала с новыми данными.


## 🔰 Задание 3

In [1]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# Функция для генерации текста с помощью BERT
def generate_text_with_bert(model, tokenizer, text, top_k=5):
    # Заменяем одно из слов на [MASK]
    masked_index = text.find(' ')
    masked_text = text[:masked_index] + " [MASK]" + text[masked_index+1:]
    
    # Кодируем текст
    input_ids = tokenizer.encode(masked_text, return_tensors="pt")
    mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]

    # Предсказываем маскированное слово
    token_logits = model(input_ids).logits
    mask_token_logits = token_logits[0, mask_token_index, :]
    
    # Выбираем top_k слов
    top_k_tokens = torch.topk(mask_token_logits, top_k, dim=1).indices[0].tolist()
    
    # Генерируем тексты, заменяя [MASK] на каждое из top_k слов
    for token in top_k_tokens:
        word = tokenizer.decode([token])
        new_text = text[:masked_index] + " " + word + text[masked_index:]
        print(f"Generated text: {new_text}")

# Загрузка модели и токенайзера для BERT (для английского языка)
tokenizer_en = BertTokenizer.from_pretrained('bert-base-uncased')
model_en = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
# Тестовый текст на английском языке
text_en = "Hello, my name is [MASK]."
generate_text_with_bert(model_en, tokenizer_en, text_en)

# Загрузка модели и токенайзера для BERT (для русского языка)
tokenizer_ru = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model_ru = BertForMaskedLM.from_pretrained('DeepPavlov/rubert-base-cased')

# Тестовый текст на русском языке
text_ru = "Привет, меня зовут [MASK]."
generate_text_with_bert(model_ru, tokenizer_ru, text_ru)

Generated text: Hello, and my name is [MASK].
Generated text: Hello, but my name is [MASK].
Generated text: Hello, " my name is [MASK].
Generated text: Hello, hi my name is [MASK].
Generated text: Hello, hello my name is [MASK].


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

C:\Users\divin\PycharmProjects\neuralnets2\venv\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\divin\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Generated text: Привет, что меня зовут [MASK].
Generated text: Привет, как меня зовут [MASK].
Generated text: Привет, теперь меня зовут [MASK].
Generated text: Привет, все меня зовут [MASK].
Generated text: Привет, почему меня зовут [MASK].


## Вывод

## 🔰 Задание 4